In [7]:
import sys
import ray
import time
import math
import os


In [8]:
ray.init()

2022-07-26 23:32:35,628	INFO services.py:1092 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '172.28.6.160',
 'raylet_ip_address': '172.28.6.160',
 'redis_address': '172.28.6.160:6379',
 'object_store_address': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-07-26_23-32-35_146553_37707',
 'metrics_export_port': 60383,
 'node_id': '1586b34fa8edaba7ad876283501ca54c33f9f528'}

In [64]:
prefix = ""
@ray.remote
def launch(cc, trace, slow_unit, ack_highprio = 1, abc_dt = 64000, abc_delta = 64000):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation"
    os.chdir(work_dir)
    print("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {} --ack_highprio {} --abc_dt {}  --abc_delta {}".format(cc, trace, slow_unit, ack_highprio, abc_dt, abc_delta))
    os.system("python run.py --c {} --trace {}  --bw 100 --topo 2layer --hpai 80 --slow_unit {} --ack_highprio {} --abc_dt {}  --abc_delta {}".format(cc, trace, slow_unit, ack_highprio, abc_dt, abc_delta))


def gen_trafic_config(wkld, nhost, load, time_s, incast, pattern=0):
    work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/traffic_gen"
    os.chdir(work_dir)
    output_name = "{}_n{}_load{}_t{}_incast{}_pattern{}".format(wkld, nhost, load, time_s, incast, pattern)
    cmd = "python traffic_gen.py -c {}.txt -n {} -l {} -b 100G -t {} -o {}.txt -i {} -p {}".format(wkld, nhost, load, time_s, output_name, incast, pattern)
    os.system(cmd)
    mix_path =  work_dir = "/home/cxyzhao/High-Precision-Congestion-Control/simulation/mix"
    os.system("cp ./{}.txt {}".format(output_name, mix_path))
    return output_name

In [ ]:
prefix = "1longflow"
lst = []
for abc_delta in [32000]:
    for abc_dt in [32000]:
        f_id = launch.remote("abc", prefix, slow_unit= 0, ack_highprio = 1, abc_dt = abc_dt, abc_delta = abc_delta)
        lst.append(f_id)
ray.wait(lst, num_returns=len(lst))        

(pid=38421) python run.py --c abc --trace 1longflow  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1 --abc_dt 32000  --abc_delta 32000


(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) 'build' finished successfully (0.374s)


(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/1longflow.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_1longflow_abcdt32000dl32000_1.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_1longflow_abcdt32000dl32000_1.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_1longflow_abcdt32000dl32000_1.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		9
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.00390625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				20Mb/s
(pid=38421) RATE_HAI			200Mb/s
(pid=38421) MIN_RATE		1000Mb/s
(pid=38421) DCTCP_RATE_AI				1000Mb/s
(pid=38421) ERROR_RATE_PER_LINK		0
(pid=38421) L2_CHUNK_SIZE			4000
(pid=38421) L2_ACK_INTERVAL			1
(pid=38421) L2_BACK_TO_ZERO			No
(

In [60]:
prefix = "1longflow"
abc_dt = 64000
abc_delta = 64000
ack = 1
lst = []
for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
    trace = prefix
    if (method != "slow"):   
        f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
        lst.append(f_id)
    else:
        f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
        lst.append(f_id)
ray.wait(lst, num_returns=len(lst))

for wkld in ["WebSearch_distribution", "FbHdp_distribution"]:
    for load in [0.3, 0.6]:
        for p in [0, 1, 2]:
            prefix = gen_trafic_config(wkld=wkld, nhost=64, load=load, time_s=0.50, incast=0, pattern=p)
            lst = []
            for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
                trace = prefix
                if (method != "slow"):   
                    f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
                    lst.append(f_id)
                else:
                    f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
                    lst.append(f_id)
            ray.wait(lst, num_returns=len(lst))
            
for wkld in ["WebSearch_distribution", "FbHdp_distribution"]:
    for load in [0.3]:
        for p in [0]:
            prefix = gen_trafic_config(wkld=wkld, nhost=64, load=load, time_s=0.50, incast=1, pattern=p)
            lst = []
            for method in ["slow", "abc", "hp", "dctcp", "timely", "dcqcn"]:
                trace = prefix
                if (method != "slow"):   
                    f_id = launch.remote(method, trace, slow_unit= 0, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
                    lst.append(f_id)
                else:
                    f_id = launch.remote("abc", trace, slow_unit=1, ack_highprio = ack, abc_dt = abc_dt, abc_delta = abc_delta)
                    lst.append(f_id)
            ray.wait(lst, num_returns=len(lst))


without incast: 13805
with incast: 13805
(pid=38421) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 1
(pid=38404) python run.py --c abc --trace WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0  --bw 100 --topo 2layer --hpai 80 --slow_unit 0 --ack_highprio 0


(pid=38421) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Entering directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38404) 'build' finished successfully (0.381s)
(pid=38421) Waf: Leaving directory `/home/cxyzhao/High-Precision-Congestion-Control/simulation/build'
(pid=38421) 'build' finished successfully (0.387s)


(pid=38421) ENABLE_QCN			Yes
(pid=38421) USE_DYNAMIC_PFC_THRESHOLD	Yes
(pid=38421) PACKET_PAYLOAD_SIZE		1000
(pid=38421) TOPOLOGY_FILE			mix/2layer.txt
(pid=38421) FLOW_FILE			mix/WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0.txt
(pid=38421) TRACE_FILE			mix/trace.txt
(pid=38421) TRACE_OUTPUT_FILE		mix/mix_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0_abc_1.tr
(pid=38421) FCT_OUTPUT_FILE		mix/fct_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0_abc_1.txt
(pid=38421) PFC_OUTPUT_FILE				mix/pfc_2layer_WebSearch_distribution_n64_load0.3_t0.1_incast0_pattern0_abc_1.txt
(pid=38421) SIMULATOR_STOP_TIME		2.1
(pid=38421) CC_MODE		9
(pid=38421) ALPHA_RESUME_INTERVAL		1
(pid=38421) RATE_DECREASE_INTERVAL		4
(pid=38421) CLAMP_TARGET_RATE		No
(pid=38421) RP_TIMER			300
(pid=38421) EWMA_GAIN			0.00390625
(pid=38421) FAST_RECOVERY_TIMES		1
(pid=38421) RATE_AI				20Mb/s
(pid=38421) RATE_HAI			200Mb/s
(pid=38421) MIN_RATE		1000Mb/s
(pid=38421) DCTCP_RATE_AI	